In [ ]:
import os

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
from skimage import io
from skimage import exposure

import gzip
import gc
import pprint as pp
import numpy as np
import pickle
import matplotlib.pyplot as plt
import logging
import random
from datetime import datetime

from keras.models import load_model

In [ ]:
log_location = "../logs/"
PATH_TO_TRAIN = "../data/input/data_stage1_train.zip"
PATH_TO_VALIDATION = "../data/validation"
PATH_TO_MODEL = "../data/models/20180214.074134.model.meta.b5.0.9507838418415674.pickle"

In [ ]:
logger = logging.getLogger()
logging.basicConfig(format="%(asctime)-15s %(message)s",
                    level=logging.DEBUG,
                    filename=os.path.join(log_location,'keras.' + datetime.now().strftime("%Y%m%d%H%M%S.%f") + '.log'))

In [ ]:
kernel_size = 16
half_kernel = int(kernel_size/2)

In [ ]:
model_config = {}

with gzip.open(PATH_TO_MODEL,'rb') as f:
        model_config = pickle.load(f)  

model = load_model(model_config['model_file_name'])

In [ ]:
images_to_process = []

for current_directory in os.listdir(PATH_TO_TRAIN):
    current_image = {}
    current_image["id"] = current_directory
    current_image["images"] = []
    current_image["masks"] = []
    for current_image_file in os.listdir(os.path.join(PATH_TO_TRAIN,current_directory,"images")):
        current_image["images"].append(os.path.join(PATH_TO_TRAIN,current_directory,"images", current_image_file))
    
    for current_image_mask in os.listdir(os.path.join(PATH_TO_TRAIN,current_directory,"masks")):
        current_image["masks"].append(os.path.join(PATH_TO_TRAIN,current_directory,"masks", current_image_mask))
   
    images_to_process.append(current_image)


In [ ]:
original_img = None
generate_images = False
last_pickled_image = None

precisions = [0.5, 0.6, 0.7, 0.8]

def is_part_of_nuclei(current_slice, precision_needed):
    current_slice_x = current_slice.reshape(1, current_slice.shape[0], current_slice.shape[1],1)
    response = model.predict(current_slice_x)
    return 1 if response[0][1] >= precision_needed else 0

for current_precision in precisions:
    for image_to_process in images_to_process:
        original_img = io.imread(image_to_process['images'][0], as_grey=True) 
        image_path = os.path.join(PATH_TO_VALIDATION,image_to_process['id'] + '.original.png')
        io.imsave(image_path,original_img)

        original_img = exposure.adjust_gamma(original_img)
        original_shape = original_img.shape
        original_img = np.pad(original_img,((half_kernel,half_kernel), (half_kernel,half_kernel)),'constant')
        masked_image_path = os.path.join(PATH_TO_VALIDATION,image_to_process['id'] + '.{}.validation.png'.format(current_precision))
        masked_image = np.zeros(original_shape)
        for current_y in range(half_kernel, original_img.shape[1] - kernel_size):
            for current_x in range(half_kernel, original_img.shape[0] - kernel_size):
                correct_current_x = current_x - half_kernel
                correct_current_y = current_y - half_kernel
                current_slice = original_img[current_x : current_x + kernel_size, current_y : current_y + kernel_size]
                is_nuclei = is_part_of_nuclei(current_slice, current_precision)
                masked_image[correct_current_x, correct_current_y] = is_nuclei
        io.imsave(masked_image_path,masked_image)
        gc.collect()
